# meta_data를  data\normalized_data.csv 에 추가하기

## meta_data 읽기 

In [1]:
import polars as pl

# Excel 파일 읽기 및 전치
df = pl.read_excel(
    r"C:\Users\Alice\OneDrive - 청주대학교\연구실 자료\연구실_노인균형프로토콜\데이터 정리-설문, 신체계측, 기타 데이터\perturb_inform.xlsm",
    sheet_name="meta"
)

# 메타데이터 항목명 추출 (None 처리)
meta_items = [f"unnamed_{i}" if item is None else str(item) for i, item in enumerate(df["subject"].to_list())]

# 전치 후 컬럼명 재지정
transpose = df.select(df.columns[1:]).transpose(include_header=True, header_name="subject")
transpose = transpose.rename(dict(zip(transpose.columns, ["subject"] + meta_items)))

# 나이 기준 age_group 생성
meta = transpose.with_columns(
    pl.when(pl.col("나이").cast(pl.Int32) < 30).then(pl.lit("young")).otherwise(pl.lit("old")).alias("age_group")
)

print(meta.select(["subject", "성별", "나이", "age_group", "주손 or 주발"]).head())
print(f"✓ 메타데이터 로드 완료: {meta.shape[0]} subjects")

shape: (5, 5)
┌─────────┬──────┬──────┬───────────┬──────────────┐
│ subject ┆ 성별 ┆ 나이 ┆ age_group ┆ 주손 or 주발 │
│ ---     ┆ ---  ┆ ---  ┆ ---       ┆ ---          │
│ str     ┆ str  ┆ str  ┆ str       ┆ str          │
╞═════════╪══════╪══════╪═══════════╪══════════════╡
│ 김연옥  ┆ F    ┆ 72   ┆ old       ┆ R            │
│ 김윤자  ┆ F    ┆ 60   ┆ old       ┆ R            │
│ 전현숙  ┆ F    ┆ 67   ┆ old       ┆ R            │
│ 한영희  ┆ F    ┆ 73   ┆ old       ┆ R            │
│ 이순우  ┆ F    ┆ 78   ┆ old       ┆ R            │
└─────────┴──────┴──────┴───────────┴──────────────┘
✓ 메타데이터 로드 완료: 61 subjects


## `meta` df 내 변수를 "data\normalized_data.csv"에 추가하기

연령변수(young/old) 생성 및 추가
주손 or 주발 변수 추가

In [2]:
# 정규화 데이터 읽기 및 메타데이터 병합
normalized = pl.read_parquet(r"C:\Users\Alice\OneDrive - 청주대학교\근전도 분석 코드\shared_files\output\03_post_processed\normalized_data.parquet")
merged = normalized.join(meta.select(["subject", "age_group", "주손 or 주발"]), on="subject", how="left")

print(merged.select(["subject", "age_group", "주손 or 주발"]).head())
print(f"✓ 병합 완료: {merged.shape}")

# Parquet 저장
merged.write_parquet(
    r"C:\Users\Alice\OneDrive - 청주대학교\근전도 분석 코드\aggregated_signal_viz\data\merged.parquet",
    compression="zstd",
    compression_level=10
)
print("✓ 저장 완료: data/merged.parquet")

shape: (5, 3)
┌─────────┬───────────┬──────────────┐
│ subject ┆ age_group ┆ 주손 or 주발 │
│ ---     ┆ ---       ┆ ---          │
│ str     ┆ str       ┆ str          │
╞═════════╪═══════════╪══════════════╡
│ 정혜진  ┆ young     ┆ R            │
│ 정혜진  ┆ young     ┆ R            │
│ 정혜진  ┆ young     ┆ R            │
│ 정혜진  ┆ young     ┆ R            │
│ 정혜진  ┆ young     ┆ R            │
└─────────┴───────────┴──────────────┘
✓ 병합 완료: (2349910, 61)
✓ 저장 완료: data/merged.parquet


In [3]:
# 병합된 subject 확인
print(f"총 {merged['subject'].n_unique()} subjects:")
print(merged["subject"].unique().to_list())

총 52 subjects:
['박진우', '안지연', '이정재', '조민석', '신향식', '김우연', '오인태', '정태웅', '이상민', '신동석', '김윤자', '이재유', '강비은', '이운복', '권유영', '최지유', '김종철', '이지수', '전현숙', '김연옥', '송석란', '이인섭', '김민정', '임남순', '방수진', '김세준', '연윤희', '이훈', '정성윤', '한영희', '임상우', '김유민', '이명희', '성효은', '한정란', '김서하', '유재원', '가윤호', '여윤숙', '오나영', '문경서', '이은수', '김병수', '최보민', '정혜진', '노선자', '김영자', '박수빈', '이순우', '방주원', '유도경', '유병한']


In [4]:
# column 확인 
print("Columns in merged data:")
print(merged.columns)

Columns in merged data:
['MocapFrame', 'MocapTime', 'DeviceFrame', 'Fx', 'Fy', 'Fz', 'Mx', 'My', 'Mz', 'Cx', 'Cy', 'Cz', 'Fx_zero', 'Fy_zero', 'Fz_zero', 'Mx_zero', 'My_zero', 'Mz_zero', 'Cx_zero', 'Cy_zero', 'Cz_zero', 'date', 'subject', 'velocity', 'trial_num', 'task', 'intervention', 'original_DeviceFrame', 'state', 'step_TF', 'RPS', 'mixed', 'platform_onset', 'platform_offset', 'step_onset', 'onset_device', 'offset_device', 'COMx', 'COMy', 'COMz', 'COMx_zero', 'COMy_zero', 'COMz_zero', 'TA', 'EHL', 'MG', 'SOL', 'PL', 'RF', 'VL', 'ST', 'RA', 'EO', 'IO', 'SCM', 'GM', 'ESC', 'EST', 'ESL', 'age_group', '주손 or 주발']


In [5]:
print(merged.select(["subject","velocity","trial_num","mixed"])\
      .filter((pl.col("subject") == '김윤자') & (pl.col("mixed") == 1))\
      .unique())

shape: (7, 4)
┌─────────┬──────────┬───────────┬───────┐
│ subject ┆ velocity ┆ trial_num ┆ mixed │
│ ---     ┆ ---      ┆ ---       ┆ ---   │
│ str     ┆ f64      ┆ i64       ┆ i64   │
╞═════════╪══════════╪═══════════╪═══════╡
│ 김윤자  ┆ 20.0     ┆ 6         ┆ 1     │
│ 김윤자  ┆ 20.0     ┆ 3         ┆ 1     │
│ 김윤자  ┆ 20.0     ┆ 2         ┆ 1     │
│ 김윤자  ┆ 20.0     ┆ 1         ┆ 1     │
│ 김윤자  ┆ 20.0     ┆ 7         ┆ 1     │
│ 김윤자  ┆ 20.0     ┆ 4         ┆ 1     │
│ 김윤자  ┆ 20.0     ┆ 5         ┆ 1     │
└─────────┴──────────┴───────────┴───────┘
